# Resample audio file and change format from wav to flac
Léa Bouffaut, Ph.D. 
-- K. Lisa Yang Center for Conservation Bioacoustics
-- lea.bouffaut@cornell.edu

In [1]:
import librosa
from scipy import signal
import os
import numpy as np
import soundfile as sf

## Create a function that will resample the audio

In [2]:
def butterworth_filter(filterspec, fs):
    """
    Designs and a butterworth filter see:
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html
    Apply as.
    trace_filtered = sp.sosfiltfilt(filter_sos, trace_original, axis=1)
    Inputs:
    :param filterspec:
    :param fs:
    Outputs:
    :return: filter_sos: a butterworth filter
    """

    filter_order, filter_critical_freq, filter_type_str = filterspec
    # Build a filter of the desired type
    wn = np.array(filter_critical_freq) / (fs / 2)  # convert to angular frequency

    filter_sos = signal.butter(filter_order, wn, btype=filter_type_str, output='sos')

    return filter_sos

def prepare_audio(x, fs, desired_fs):
    # 1) the signal is filtered
    # Butterworth low-pass filter to remove everything above desired_fs
    # If we have to down-sample, the data is bandpass filtered instead
    if desired_fs < fs:
        sos = butterworth_filter([3, [5, (desired_fs / 2)], 'bandpass'], fs)

    else:
        sos = butterworth_filter([3, 5, 'hp'], fs)

    x = signal.sosfiltfilt(sos, x)

    # 2) Resample
    x = librosa.resample(
        x.T, orig_sr=fs, target_sr=desired_fs)
    fs = desired_fs

    return x, fs

## Define the paths of orginal audio and new audio
The original_audio folder contained a 1 min .wav file sampled at 48 kHz.
We want to downsample it to 22 kHz and save it as a .flac file.

In [3]:
original_fs_audio_folder = 'original_audio/'
new_fs_audio_folder = 'resampled/'                    

(optional) -- Run cell below to make sure we're analyzing the correct folder - it should print the name of the first .wav file in the folder

In [4]:
for file in os.listdir(original_fs_audio_folder):
    if file.endswith(".wav"):
        print(os.path.join(original_fs_audio_folder, file))
    break

original_audio/KillerWhale_Skjervøy_48k_20200110-110944.wav


Define the new sampling frequency

In [5]:
desired_fs = 22000 #Hz
original_file_format = '.wav'
desired_file_format = '.flac'

## Resample all audio files and save

In [6]:
for file in os.listdir(original_fs_audio_folder):
    if file.endswith(original_file_format):
        print('Processing: ' + os.path.join(original_fs_audio_folder, file))
        
        # Load the audio file
        x, fs = librosa.load(os.path.join(original_fs_audio_folder, file), sr=None, mono=False)
        
        # Resample the audio file
        x, fs_new = prepare_audio(x, fs, desired_fs)
        
        # Save the audio file
        if desired_fs > 1000:
            filename = os.path.join(new_fs_audio_folder, file[0:-4]+'_resampled_'+ str(int(desired_fs/1000))+'k' + desired_file_format)
        else:
            filename = os.path.join(new_fs_audio_folder, file[0:-4]+'_resampled_'+ str(int(desired_fs))+ desired_file_format)

        sf.write(filename, x, fs_new)
        print('Saved resampled: ' + filename)

Processing: original_audio/KillerWhale_Skjervøy_48k_20200110-110944.wav
Saved resampled: resampled/KillerWhale_Skjervøy_48k_20200110-110944_resampled_22k.flac
